# conversion

> Unit and time conversion

In [ ]:
#| default_exp conversion

In [ ]:
#| export
import math

In [ ]:
#| export
AU_METER = 149597870700
LY_METER = 9.461E+15
PS_METER = 3.086E+16

In [ ]:
#| export
def dms_to_decimal(degree, minute, second):
    """
    Conversion from DMS format (e.g. 15°42'9") to decimal format (e.g. 15.7025).
    """
    sign = -1 if degree < 0 else 1
    degree = abs(degree)
    minute += second / 60
    decimal = degree + (minute / 60)
    return round(decimal * sign, 4)

#| export
def decimal_to_dms(decimal):
    """
    Conversion from decimal format (e.g. 15.7025) to DMS format (e.g. 15°42'9").
    """
    sign = -1 if decimal < 0 else 1
    decimal = abs(decimal)
    degree = int(decimal)
    frac, _ = math.modf(decimal)
    frac *= 60
    minute = int(frac)
    frac, _ = math.modf(frac)
    second = 60 * frac
    return degree * sign, minute, round(second, 4)

In [ ]:
assert dms_to_decimal(15, 42, 9) == 15.7025
assert decimal_to_dms(15.7025) == (15, 42, 9)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

15.7025